In [32]:
#load packages
#.libPaths("/local/home/katrinac/miniconda3/envs/r4.0/lib/R/library") #rgdal won't work with the older version of R, so specify libpaths to the more current R version 
Packages <- c("tidyr", "dplyr", "data.table", "lubridate", "broom")
invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

#define functions
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0

#read in data
SimConnMeta <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableWithMeta.csv") #this is all of the simulated data plus the meta data for each site
#GenSimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/GenSimConnectivityFullTable.csv")[, -"AnnRecPart"] #this is the simulated data AND the observations of parentage matches and other field surveying data
#rename the data.table so that columns names are clear and verbose!
#setnames(GenSimConn, c("SourceSampled", "DestSampled", "SimMonth", "SimDay", "SimYear", "YearSampled", "SimMonsoon", "DailyParticles", "ParticlesReleasedDaily", "SourcePropSamp", "DestPropSamp"), c("source_sampled", "dest_sampled", "sim_month", "sim_day", "sim_year", "year_sampled", "sim_monsoon", "daily_particles_recruited", "daily_particles_released", "source_prop_samp", "dest_prop_samp") )

#read in the discretized dispersal kernel connectivity matrices from the genetics
#GenConnMat <- fread(file="~/DispersalVariation/data/script_output/final_results/dispersal_route_correlation/KernelConnMatrices.csv")[, -c("prob_disp15", "prob_disp16", "prob_disp17", "prob_disp18")] #drop the years not coinciding with model  ]
#setnames(GenConnMat, c("org_site", "dest_site", "prob_disp12", "prob_disp13", "prob_disp14"), c("source", "destination", "2012", "2013", "2014"))

#read in environmental data
ENSO<- fread("~/oceanography/empirical_data/environment/enso/meiv2.csv")
#reformat to match month ID in ROMs data. MEI has overlapping 2 month seasons
setnames(ENSO, c("YEAR","DJ", "JF", "FM", "MA", "AM", "MJ", "JJ", "JA", "AS", "SO", "ON", "ND"), c("year","12,1", "1,2", "2,3", "3,4", "4,5", "5,6", "6,7", "7,8", "8,9", "9,10", "10,11", "11,12"))

#read in the centroids adjusted for the simulation, so the Magbangons combined 
centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv")
setorder(centroids, site)

UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other", "CAI") 


In [13]:
head(Enso)

year,mei,month
1979,0.47,12
1979,0.47,1
1980,0.35,12
1980,0.35,1
1981,-0.33,12
1981,-0.33,1


In [ ]:
##add in the discretized kernel connectivity matrix from the genetics
##reformat to be long form and fixed discrepancies in Magbangon names
#GenConnMat <- unique(melt(GenConnMat, id.vars=c("source", "destination"), variable.name="year", value.name="prob_disp")[#make long format
#    grep("N. Magbangon|S. Magbangon", GenConnMat$source), source :="Magbangon"][ #replace the S/N Magbangons with just Magbangon in the source column
#    grep("N. Magbangon|S. Magbangon", GenConnMat$destination), destination :="Magbangon"][ #then the destination
#    , prob_disp := sum(prob_disp), by=c("source", "destination", "year")], by=c("source", "destination", "year"))[#finally aggregate and sum the data by source/dest and only the rows with Magbangons will be collapsed into one upon using unique to get rid of repeats
#    , year := as.numeric(as.character(year))]#melt turns year into a factor for a reason I don't understand. change back to a number
#                                                                                                                  #note to self, I checked this code worked as expected by making sure the combined Magbangon prob_disp values added to what the prob_disp column ends up saying after the unique()
#
#GenSimConn <- GenConnMat[GenSimConn, on=.(source, destination, year=year_sampled)] #join discretized kernels into sim connectivity table
#setnames(GenSimConn, "year", "year_sampled")
##for each source, what is the normalized recruitment at each destination? 
#GenSimConn[, annual_source_normalized_recruitment := sum(daily_particles_recruited)/sum(daily_particles_released), by=c("source", "destination","year_sampled")]
#
##check that it worked as expected 
#sum(unique(GenSimConn[source=="Sitio Baybayon" & year_sampled ==2014], by=c("source", "destination"))[, annual_source_normalized_recruitment]) #should equal 1




In [37]:
#add in all of the physical environmental data
#read in environmental data
ENSO<- fread("~/oceanography/empirical_data/environment/enso/meiv2.csv")
#reformat to match month ID in ROMs data. MEI has overlapping 2 month seasons
setnames(ENSO, c("YEAR","DJ", "JF", "FM", "MA", "AM", "MJ", "JJ", "JA", "AS", "SO", "ON", "ND"), c("year","12,1", "1,2", "2,3", "3,4", "4,5", "5,6", "6,7", "7,8", "8,9", "9,10", "10,11", "11,12"))

#add in the ENSO data
#convert to long form
ENSO <- melt(ENSO, id.vars=c("year"), variable.name="month", value.name="mei")[#make long form with month id column to house what were column headers
    , lapply(.SD, function(x) unlist(tstrsplit(x, ","))), #split the comma separated month numbers into separate rows
   .SDcols = "month",by = c("year","mei")][, month := as.numeric(month)] #change month column from character to number

#ENSO <- unique(ENSO, by=c("year", "month"))      #mei is calculated in 2 month overlapping windows... not sure how to deal with this if MEI is two values for one month. I will just pick the first row and be consistent with using that, but follow up later with others to think if that is fine.

nrow(ENSO) #should be 504
ENSO <- ENSO[year %in% c(2011, 2012, 2013, 2014)][, annual_avg_mei := mean(mei), by=c("year")][
    , monsoon := ifelse(month %in% c(11, 12, 1, 2, 3, 4, 5), "NEM", "SWM")][
    , annual_monsoon_avg_mei := mean(mei), by=c("year", "monsoon")][, monsoon_avg_mei := mean(mei), by=c("monsoon")][ 
    , annual_monsoon_max_mei := max(mei), by=c("year","monsoon")][
    , annual_monsoon_min_mei := min(mei), by=c("year","monsoon")]
ConnPhysMeta <- right_join(ENSO, SimConnMeta, by=c(year="sim_year", month="sim_month"))[source %!in% UnsampledSites & dest %!in% UnsampledSites]

[1] 1008

In [39]:
head(ENSO)
head(SimConnMeta)
head(ConnPhysMeta[year != 2010])

year,mei,month,annual_avg_mei,monsoon,annual_monsoon_avg_mei,monsoon_avg_mei,annual_monsoon_max_mei,annual_monsoon_min_mei
2011,-1.83,12,-1.3400000,NEM,-1.5235714,-0.61625,-1.08,-1.83
2011,-1.83,1,-1.3400000,NEM,-1.5235714,-0.61625,-1.08,-1.83
2012,-1.08,12,-0.3200000,NEM,-0.4807143,-0.61625,-0.07,-1.08
2012,-1.08,1,-0.3200000,NEM,-0.4807143,-0.61625,-0.07,-1.08
2013,-0.05,12,-0.4116667,NEM,-0.3385714,-0.61625,-0.05,-1.18
2013,-0.05,1,-0.4116667,NEM,-0.3385714,-0.61625,-0.05,-1.18


date,source,dest,sim_month,sim_day,sim_year,year_sampled,sim_monsoon,daily_particles_rec,daily_particles_released,dist_km,bearing,direction
2010-10-01,Other,Palanas,10,10,2010,2011,SWM,9,4497728,NA,NA,NA
2010-10-01,Other,Wangag,10,10,2010,2011,SWM,4,4497728,NA,NA,NA
2010-10-01,Other,Magbangon,10,10,2010,2011,SWM,11,4497728,NA,NA,NA
2010-10-01,Other,Cabatoan,10,10,2010,2011,SWM,12,4497728,NA,NA,NA
2010-10-01,Other,Caridad Cemetery,10,10,2010,2011,SWM,0,4497728,NA,NA,NA
2010-10-01,Other,Caridad Proper,10,10,2010,2011,SWM,1,4497728,NA,NA,NA


year,mei,month,annual_avg_mei,monsoon,annual_monsoon_avg_mei,monsoon_avg_mei,annual_monsoon_max_mei,annual_monsoon_min_mei,date,source,dest,sim_day,year_sampled,sim_monsoon,daily_particles_rec,daily_particles_released,dist_km,bearing,direction
2011,-1.83,12,-1.34,NEM,-1.523571,-0.61625,-1.08,-1.83,2011-12-01,Sitio Baybayon,Palanas,12,2012,NEM,0,1984,27.29194,-14.977111,345.0229
2011,-1.83,12,-1.34,NEM,-1.523571,-0.61625,-1.08,-1.83,2011-12-01,Sitio Baybayon,Wangag,12,2012,NEM,0,1984,26.86207,-14.278206,345.7218
2011,-1.83,12,-1.34,NEM,-1.523571,-0.61625,-1.08,-1.83,2011-12-01,Sitio Baybayon,Magbangon,12,2012,NEM,32,1984,25.11346,-12.688177,347.3118
2011,-1.83,12,-1.34,NEM,-1.523571,-0.61625,-1.08,-1.83,2011-12-01,Sitio Baybayon,Cabatoan,12,2012,NEM,0,1984,24.91056,-12.353326,347.6467
2011,-1.83,12,-1.34,NEM,-1.523571,-0.61625,-1.08,-1.83,2011-12-01,Sitio Baybayon,Caridad Cemetery,12,2012,NEM,0,1984,22.43538,-8.054163,351.9458
2011,-1.83,12,-1.34,NEM,-1.523571,-0.61625,-1.08,-1.83,2011-12-01,Sitio Baybayon,Caridad Proper,12,2012,NEM,0,1984,21.75097,-7.170156,352.8298


In [41]:
test_mod <- glm(data=ConnPhysMeta, daily_particles_rec ~ as.factor(year), family="poisson")

In [42]:
summary(test_mod)


Call:
glm(formula = daily_particles_rec ~ as.factor(year), family = "poisson", 
    data = ConnPhysMeta)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
 -2.614   -2.028   -1.843   -1.590  113.871  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)          0.441482   0.004400  100.33   <2e-16 ***
as.factor(year)2011  0.279416   0.004705   59.39   <2e-16 ***
as.factor(year)2012  0.088072   0.004765   18.48   <2e-16 ***
as.factor(year)2013 -0.207551   0.004886  -42.48   <2e-16 ***
as.factor(year)2014  0.787378   0.004808  163.77   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 8912114  on 638247  degrees of freedom
Residual deviance: 8785250  on 638243  degrees of freedom
AIC: 8998284

Number of Fisher Scoring iterations: 9


In [ ]:

head(GenSimConn)

In [ ]:
#add in MARSPEC data
test <- list_layers(marine=TRUE)
setDT(test)

test[ end_year >=2014]

In [ ]:
bath_mean <- load_layers("BO_bathymean", datadir = "~/oceanography/empirical_data/environment/") 
bath_mean
max_vel <- load_layers("BO2_curvelmax_bdmin", datadir = "~/oceanography/empirical_data/environment/")
max_vel
sst <- load_layers("BO21_tempmean_ss" , datadir = "~/oceanography/empirical_data/environment/")
sst

In [ ]:
sst$year

In [ ]:
#geographic extent of our area
extent <-c(min(centroids$lon), max(centroids$lon), min(centroids$lat), max(centroids$lat))

In [ ]:
install.packages("oce", dependencies = TRUE)

In [ ]:
library("ncdf4")

In [ ]:
nc <- nc_open("~/oceanography/empirical_data/environment/noaa_sst/sst.wkmean.1990-present.nc")

In [ ]:
print(nc)

In [ ]:
sst <- data.table(lat = ncvar_get(nc, "lat"),
lon = ncvar_get(nc, "lon"),
time = ncvar_get(nc, "time"),
sst = ncvar_get(nc, "sst"))[, time := as.POSIXct(time, origin = '1981-01-01 00:00:00', tz = 'UTC')]


In [ ]:
summary(sst$time)

In [ ]:

SST_DT <- setDT(matrix(nrow=0, ncol=4))

cols_conn_mat <- c("date","lat", "lon", "weekly_mean")
cols_tmp <- c("lat", "lon", "weekly_mean")

names(conn_mat_full) <- cols_conn_mat

for(i in 1:nrow(all_dates2)){
    
    index_eval <- all_dates2$index[i]
    
    tmp <- as.data.frame(conn_mat_all[[i]], xy=T)   #pull out a data frame for each date
    names(tmp) <- cols_tmp
    
    date <- filter(all_dates2, index==index_eval)[2] #grab the date
    date_df <- do.call("rbind", replicate(784, date, simplify = FALSE)) #make a df of the date
    tmp2 <- bind_cols(date_df, tmp)

 
    conn_mat_full <- rbind(conn_mat_full, tmp2) 
}
